In [ ]:
from utils import load_data

seq_arr, test_seq_arr, labels, word_index, index_word, vs, embedding_matrix = load_data('word', 'glove')
seq_arr.shape, test_seq_arr.shape, embedding_matrix.shape

In [58]:
from utils import *
from keras import callbacks
from timeit import default_timer as timer
from keras import models, losses, metrics, layers, optimizers
from keras.utils import multi_gpu_model
import tensorflow as tf
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os
import keras

import numpy as np
import pandas as pd

from tensorflow.python.client import device_lib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

datadir = '/home/wjk68/data/quora/'

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


get_available_gpus()

['/device:GPU:0', '/device:GPU:1']

In [ ]:
sequences, labels, test_sequences, word_index, index_word, vs = retrieve_saved_data(train_df, test_df,
                                                                                    filters = '')
print('Shape of data: ', sequences.shape)
print('Shape of labels: ', labels.shape)
print('Number of words in vocab: ', vs)

## Word-Level Model

In [ ]:
from keras.models import *
from keras.layers import *

sequence_input = Input(shape=(sequences.shape[1],), dtype='int32')

embedding_layer = Embedding(embedding_matrix.shape[0], 
                             embedding_matrix.shape[1],
                             weights = [embedding_matrix],
                             name = 'embedding')

embedded_sequences = embedding_layer(sequence_input)

l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(5)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
l_dense = Dropout(0.5)(l_dense)
preds = Dense(1, activation='sigmoid')(l_dense)
                          
model = Model(inputs = [sequence_input], output = [preds])

model.compile(optimizer = optimizers.Adam(), loss = losses.binary_crossentropy,
              metrics = [metrics.binary_crossentropy, metrics.binary_accuracy,
                         f1])

model.summary()

In [ ]:
history = model.fit(sequences, labels, validation_split = 0.5,
          epochs = 25, batch_size = 256)

In [64]:
EMBEDDING_FILE = datadir + 'glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

embedding_matrix = np.random.normal(emb_mean, emb_std, (vs, embed_size))

print('Embedding Matrix Shape: ', embedding_matrix.shape)

In [69]:
from multiprocessing import Pool
from timeit import default_timer as timer
pool = Pool(15)

def get_embed_vector(word):
    return embeddings_index.get(word)

Process ForkPoolWorker-32:
Process ForkPoolWorker-26:
Process ForkPoolWorker-38:
Process ForkPoolWorker-43:
Process ForkPoolWorker-25:
Process ForkPoolWorker-41:
Process ForkPoolWorker-44:
Process ForkPoolWorker-24:
Process ForkPoolWorker-42:
Process ForkPoolWorker-40:
Process ForkPoolWorker-37:
Process ForkPoolWorker-23:
Process ForkPoolWorker-29:
Process ForkPoolWorker-45:
Process ForkPoolWorker-30:
Process ForkPoolWorker-28:
Process ForkPoolWorker-27:
Process ForkPoolWorker-39:
Process ForkPoolWorker-22:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/m

  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/p

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/local/gcc-6_3_0/openmpi-2_0_1/python/3.6.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter_

In [70]:
words = word_index.keys()
r = pool.map(get_embed_vector, words)
pool.close()
pool.join()

# not_in = 0
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None: 
#         embedding_matrix[i] = embedding_vector
#     else:
#         not_in += 1

# print('There were ', not_in 'words not in the embeddings.')

In [76]:
embedding_matrix = np.random.normal(emb_mean, emb_std, (vs, embed_size))

print('Embedding Matrix Shape: ', embedding_matrix.shape)

NameError: name 'emb_mean' is not defined

In [77]:
arr = np.array(r)

In [82]:
not_found = 0
for a in arr:
    if a is not None:
        continue
    else:
        not_found += 1

In [83]:
not_found

3132

In [84]:
embedding_matrix = np.random.normal(size = (vs, 300))
for i, a in enumerate(arr):
    if a is not None:
        embedding_matrix[i] = a
    else:
        continue
        
embedding_matrix.shape

(59728, 300)

In [85]:
np.save('glove_embeddings.npy', embedding_matrix)

In [74]:
for i in r:
    if i == None:
        print('Found')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [73]:
np.mean(r == None)

0.0

In [67]:
with open('word_index.json', 'w') as f:
    f.write(json.dumps(word_index))
    
with open('index_word.json', 'w') as f:
    f.write(json.dumps(index_word))

1068893

1188349

In [66]:
sequences, labels, test_sequences, word_index, index_word, wc, vs = format_data(train_df, test_df,
                                                                            filters = '')
print('Shape of data: ', sequences.shape)
print('Shape of labels: ', labels.shape)
print('Number of words in vocab: ', vs)

Shape of data:  (1306122, 503)
Shape of labels:  (1306122,)
Number of words in vocab:  59728


In [54]:
def sequence_to_query(sequence, index_word):
    query = [index_word.get(i, 'UNK') for i in sequence]
    return ' '.join(query)

In [56]:
index_word[0]

KeyError: 0

In [55]:
sequence_to_query(sequences[15], index_word)

'UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK

In [57]:
fasttext_embeddings = np.load('fasttext_embeddings.npy')
# fasttext_embeddings = normalize(fasttext_embeddings)